# Restoration and degradation widget
##### The purpose of this notebook is to format the Typology_and_Restoration_Potential table to produce tables that can be used by the API. The data models we need should have the following format:

**Restoration potential:**  
{  
location_id [str]  
indicator [str] - restorable_area, mangrove_area, restoration_potential_score  
value [numeric]  
unit [str] - ha, %  
}  

where:

restorable_area = Tot_Restor  
mangrove_area  = Total Mangrove Area 2020 (ha) 
restoration_potential_score  =  Mean_Score  

**Degradation:**  
{  
location_id [str]  
indicator [str] - degraded_area, lost_area, main_loss_driver  
value [numeric]  
unit [str] - ha, %  
}  

where:

degraded_area = Rest_Area_Dgrd  
lost_area = Area_loss_ha  
main_loss_driver = Loss_Driver


### Steps:
1. Import packages and data
2. Format restoration potential table:
- Get countries for the table
- Give each country their corresponding iso code
- Add staging ids
- Create indicators
- Table cleaning
- Save table
3. Format degradation table:
- Get countries for the table
- Give each country their corresponding iso code
- Add staging ids
- Create indicators
- Table cleaning
- Save table

#### 1. Import packages and data

In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import fiona


In [4]:
# Import geodatabase and get Global_Mangrove_Restoration_Jurisdiction_Envelope
#path_in = '../../../../data/Typology_and_Restoration_Potential/Data'
#gdb_file =f'{path_in}/MOW_Global_Mangrove_Restoration_20190411.gdb'
#layers = fiona.listlayers(gdb_file)
#layer0 = gpd.read_file(gdb_file, driver='FileGDB', layer=0)

path_in = 'https://storage.googleapis.com/mangrove_atlas/widget_data'
data_df = pd.read_csv(f'{path_in}/Vizz_Country_Stats_Dec2022.csv', index_col=0)
data_df.loc[data_df['Total Area Restorable (ha)'].isna(), 'Total Area Restorable (ha)'] = 0
data_df.head()


,Name,Total Mangrove Area 2020 (ha),Total Area of Loss (ha),Total Area Restorable (ha),OVERALL_SRE,Total_Fish,Fish,Crab,Bivalve,Shrimp,Total AGC (mtCO2e) from restored areas,Total SOC (mtCO2e) from restored areas,Proportion_Restorable,Extreme_Weather,Non_Productive,Commodities,Driver
1,American Samoa,32.004317,0.594951,0.442066,0.830489,9.129377e+03,9081.125331,4.825178e+01,0.0,0.0,82.919195,144.531066,1.38,0.000619,0.000594,0.003208,Commodities
2,Angola,28319.919690,1546.820441,856.958013,0.692000,4.553915e+04,45539.145920,0.000000e+00,0.0,0.0,110509.432500,234265.161100,3.03,0.254498,8.113579,0.000000,Non-Productive Conversion
3,Anguilla,3.836882,1.571377,1.571377,0.620000,2.378690e+04,10165.451040,1.362145e+04,0.0,0.0,11.015105,461.861506,40.95,0.000000,0.015714,0.000000,Non-Productive Conversion
4,Antigua and Barbuda,871.859248,107.809050,107.809050,0.632000,2.045093e+06,873980.752300,1.171113e+06,0.0,0.0,2714.655158,28326.766570,12.37,0.000000,1.078090,0.000000,Non-Productive Conversion
5,Aruba,47.371025,8.345105,5.643541,0.597168,8.612742e+04,36806.980660,4.932044e+04,0.0,0.0,185.684321,1142.345668,11.91,0.016808,0.034860,0.004767,Non-Productive Conversion


### Add (staging) locations

In [5]:
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country']
locations.head()

,name,iso,type,area_m2,wdpaid,globalid,perimeter_m,location_idn,coast_length_m,geometry
82,Qatar,QAT,country,3.880224,NaN,{AF97ABE2-6405-4438-A7ED-1494A43DA379},8.392644,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd,1345769.96,"MULTIPOLYGON (((50.73769 24.93464, 50.73779 24..."
89,Mayotte,MYT,country,5.611808,NaN,{57E86B5B-7EF0-4754-A8D4-A9DC3212D421},10.086238,0750953f-4af9-549b-aeea-329663249a56,291036.71,"POLYGON ((46.63483 -12.96039, 46.63197 -12.969..."
118,Vietnam,VNM,country,90.156489,NaN,{B2A84FBB-34CD-4A51-9463-B9DB2DB62A10},81.714911,09a1ab14-11ad-56ec-8acb-a149e5697abd,9005760.08,"MULTIPOLYGON (((104.31952 10.36051, 104.31975 ..."
132,Grenada,GRD,country,2.154728,NaN,{F8753179-5FFA-4D9E-8AD9-083F31C48528},6.743601,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e,260664.47,"MULTIPOLYGON (((-61.91525 11.37330, -61.91813 ..."
149,India,IND,country,473.029671,NaN,{A4A6CE4D-8D03-4246-9A2F-BD9811232115},211.564078,0c07ca53-7b17-5650-a2c6-0cc27249a4bd,16917891.22,"MULTIPOLYGON (((79.52922 9.38411, 79.52921 9.3..."


In [6]:
api_locs = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/locations_staging.csv')
api_locs.rename(columns={'location_id': 'location_idn'}, inplace=True)
api_locs.head()

,id,location_idn
0,1563,000bd204-c0fd-510b-a1ad-132a7ef7470d
1,1564,00250a0f-f66d-54a0-b7a3-d80035881cbf
2,1565,0041637b-f6a2-5b89-87ce-850f5c5431b3
3,1566,005b49ef-6b7f-575a-85b3-ff19261a0755
4,1567,00921349-70fb-5a7e-8207-b3157aecc349


In [7]:
# Import gadm file to retrieve ISO codes
#gadm = gpd.read_file('../../../../data/gadm36_level0_original/gadm36_level0_original.shp')

#Get ISO codes from locations file
isocodes = locations[['name', 'iso']].copy()
isocodes.head()

,name,iso
82,Qatar,QAT
89,Mayotte,MYT
118,Vietnam,VNM
132,Grenada,GRD
149,India,IND


#### 2. Create Restoration Potential table
Get countries for the table


In [8]:
rp = data_df[['Name']]
rp.head()

,Name
1,American Samoa
2,Angola
3,Anguilla
4,Antigua and Barbuda
5,Aruba


Give each country their corresponding iso code


In [9]:
# Add iso codes to countries in rp dataframe
rp = pd.merge(rp, isocodes, how='left', left_on='Name', right_on='name')
rp.head()

,Name,name,iso
0,American Samoa,American Samoa,ASM
1,Angola,Angola,AGO
2,Anguilla,Anguilla,AIA
3,Antigua and Barbuda,Antigua and Barbuda,ATG
4,Aruba,Aruba,ABW


In [10]:
# Check which countries did not match with gadm and have no iso
[rp['iso'].isnull()]

[0      False
 1      False
 2      False
 3      False
 4      False
        ...  
 117    False
 118    False
 119    False
 120    False
 121    False
 Name: iso, Length: 122, dtype: bool]

In [11]:
locations[locations['name'].str.contains('Ivo')][['name', 'iso']]

,name,iso
1969,Côte d'Ivoire,CIV


In [12]:
# Add isocodes to those countries
rp.loc[rp.Name == 'Mexico', 'iso'] = 'MEX'
rp.loc[rp.Name == 'Cote d\'Ivoire', 'iso'] = 'CIV'
rp.loc[rp.Name == 'Congo', 'iso'] = 'COG'
rp.loc[rp.Name == 'Sao Tome and Principe', 'iso'] = 'STP'

In [13]:
rp = rp.drop(columns='name')
rp.head()

,Name,iso
0,American Samoa,ASM
1,Angola,AGO
2,Anguilla,AIA
3,Antigua and Barbuda,ATG
4,Aruba,ABW


Add staging ids

In [14]:
# Add codes from locations
rp_locs = pd.merge(rp, locations[['iso', 'location_idn']], on='iso', how='left')
rp_locs = pd.merge(rp_locs, api_locs, on='location_idn', how='left')
#mang_df_final = mang_df_final[mang_df_final.id.notna()]
rp_locs.drop(columns=['location_idn'], inplace=True)
rp_locs

,Name,iso,id
0,American Samoa,ASM,2346
1,Angola,AGO,2029
2,Anguilla,AIA,1915
3,Antigua and Barbuda,ATG,3095
4,Aruba,ABW,2707
...,...,...,...
117,Vietnam,VNM,1681
118,British Virgin Islands,VGB,3037
119,"Virgin Islands, U.S.",VIR,2339
120,Wallis and Futuna,WLF,4170


In [15]:
# Countries with no ID
rp_locs[rp_locs['id'].isnull()]

,Name,iso,id


Create indicators

In [16]:
data_df.columns

Index(['Name', 'Total Mangrove Area 2020 (ha)', 'Total Area of Loss (ha)',
       'Total Area Restorable (ha)', 'OVERALL_SRE', 'Total_Fish', 'Fish',
       'Crab', 'Bivalve', 'Shrimp', 'Total AGC (mtCO2e) from restored areas',
       'Total SOC (mtCO2e) from restored areas', 'Proportion_Restorable',
       'Extreme_Weather', 'Non_Productive', 'Commodities', 'Driver'],
      dtype='object')

In [17]:
# Add restorable area indicator
ind1 = rp_locs.copy()
ind1['indicator'] = 'restorable_area'
ind1 = pd.merge(ind1,data_df[['Name','Total Area Restorable (ha)']],on='Name', how='left').rename(columns={'Total Area Restorable (ha)':'value'})
ind1['unit']='ha'
ind1.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,restorable_area,0.442066,ha
1,Angola,AGO,2029,restorable_area,856.958013,ha
2,Anguilla,AIA,1915,restorable_area,1.571377,ha
3,Antigua and Barbuda,ATG,3095,restorable_area,107.809050,ha
4,Aruba,ABW,2707,restorable_area,5.643541,ha


In [18]:
# Add mangrove area indicator
ind2 = rp_locs.copy()
ind2['indicator'] = 'mangrove_area'
ind2 = pd.merge(ind2,data_df[['Name','Total Mangrove Area 2020 (ha)']],on='Name', how='left').rename(columns={'Total Mangrove Area 2020 (ha)':'value'})
ind2['unit']='ha'
ind2.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,mangrove_area,32.004317,ha
1,Angola,AGO,2029,mangrove_area,28319.919690,ha
2,Anguilla,AIA,1915,mangrove_area,3.836882,ha
3,Antigua and Barbuda,ATG,3095,mangrove_area,871.859248,ha
4,Aruba,ABW,2707,mangrove_area,47.371025,ha


In [19]:
# Add restoration potential score indicator
ind3 = rp_locs.copy()
ind3['indicator'] = 'restoration_potential_score'
ind3 = pd.merge(ind3,data_df[['Name','OVERALL_SRE']],on='Name', how='left').rename(columns={'OVERALL_SRE':'value'})
ind3['value'] = round(ind3['value']*100, 0)
ind3['unit']=''
ind3.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,restoration_potential_score,83.0,
1,Angola,AGO,2029,restoration_potential_score,69.0,
2,Anguilla,AIA,1915,restoration_potential_score,62.0,
3,Antigua and Barbuda,ATG,3095,restoration_potential_score,63.0,
4,Aruba,ABW,2707,restoration_potential_score,60.0,


In [20]:
# Merge all indicators in same table
restoration = pd.concat([ind1, ind2, ind3], ignore_index=True)
restoration


,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,restorable_area,0.442066,ha
1,Angola,AGO,2029,restorable_area,856.958013,ha
2,Anguilla,AIA,1915,restorable_area,1.571377,ha
3,Antigua and Barbuda,ATG,3095,restorable_area,107.809050,ha
4,Aruba,ABW,2707,restorable_area,5.643541,ha
...,...,...,...,...,...,...
361,Vietnam,VNM,1681,restoration_potential_score,76.000000,
362,British Virgin Islands,VGB,3037,restoration_potential_score,60.000000,
363,"Virgin Islands, U.S.",VIR,2339,restoration_potential_score,60.000000,
364,Wallis and Futuna,WLF,4170,restoration_potential_score,NaN,


In [21]:
country_name = "Indonesia"
restoration[restoration['Name'].str.contains(country_name)]

,Name,iso,id,indicator,value,unit
54,Indonesia,IDN,3384,restorable_area,2.045362e+05,ha
176,Indonesia,IDN,3384,mangrove_area,2.953651e+06,ha
298,Indonesia,IDN,3384,restoration_potential_score,7.900000e+01,


In [22]:
restoration[['indicator', 'value']].groupby('indicator').sum()

,value
indicator,
mangrove_area,1.472742e+07
restorable_area,8.183090e+05
restoration_potential_score,7.877000e+03


Table cleaning

In [23]:
# Countries with no staging id
restoration[restoration.id.isna()]

,Name,iso,id,indicator,value,unit


In [24]:
restoration_final = restoration [['id', 'indicator', 'value', 'unit']].copy()
restoration_final.unit = 'ha'
restoration_final['year'] = 2020
restoration_final.rename(columns={'id':'location_id'}, inplace=True)
restoration_final.head(2)


,location_id,indicator,value,unit,year
0,2346,restorable_area,0.442066,ha,2020
1,2029,restorable_area,856.958013,ha,2020


In [25]:
mangrove_original = sum(data_df[data_df['Total Mangrove Area 2020 (ha)'].notna()]['Total Mangrove Area 2020 (ha)'])
mangrove_final = sum(restoration_final[restoration_final['indicator'] == 'mangrove_area']['value'])

restorable_original = sum(data_df[data_df['Total Area Restorable (ha)'].notna()]['Total Area Restorable (ha)'])
restorable_final = sum(restoration_final[restoration_final['indicator'] == 'restorable_area']['value'])

ta =  mangrove_original == mangrove_final
re =  restorable_original == restorable_final 

print('Original and final numbers check')
if(ta):
    print(f'Total mangrove area: {ta}')
    print(f'\tFinal area: {mangrove_final}')
else:
    print(f'\tOriginal area: {mangrove_original}')
    print(f'\tFinal area: {mangrove_final}')

if(re):
    print(f'Restorable: {re}')
    print(f'\tFinal restorable: {restorable_final}')
else:
    print(f'\tOriginal restorable: {restorable_original}')
    print(f'\tFinal restorable: {restorable_final}')

Original and final numbers check
Total mangrove area: True
	Final area: 14727416.488607474
Restorable: True
	Final restorable: 818309.0392280952


In [26]:
restoration_final[restoration_final['indicator'].notna()]

,location_id,indicator,value,unit,year
0,2346,restorable_area,0.442066,ha,2020
1,2029,restorable_area,856.958013,ha,2020
2,1915,restorable_area,1.571377,ha,2020
3,3095,restorable_area,107.809050,ha,2020
4,2707,restorable_area,5.643541,ha,2020
...,...,...,...,...,...
361,1681,restoration_potential_score,76.000000,ha,2020
362,3037,restoration_potential_score,60.000000,ha,2020
363,2339,restoration_potential_score,60.000000,ha,2020
364,4170,restoration_potential_score,NaN,ha,2020


Save table

In [47]:
restoration_final[~restoration_final.location_id.isna()].to_csv('../../../../data/RestorationPotential_202212.csv', index = False)

## Format degradation table


Create indicators

In [24]:
data_df.columns

Index(['Name', 'Total Mangrove Area 2020 (ha)', 'Total Area of Loss (ha)',
       'Total Area Restorable (ha)', 'OVERALL_SRE', 'Total_Fish', 'Fish',
       'Crab', 'Bivalve', 'Shrimp', 'Total AGC (mtCO2e) from restored areas',
       'Total SOC (mtCO2e) from restored areas', 'Proportion_Restorable',
       'Extreme_Weather', 'Non_Productive', 'Commodities', 'Driver'],
      dtype='object')

In [25]:
# Create degraded area indicator
ind1 = rp_locs.copy()
ind1['indicator'] = 'degraded_area'
#Degraded area = Loss - Restorable
ind1 = pd.merge(ind1,data_df[['Name','Total Area of Loss (ha)', 'Total Area Restorable (ha)']],on='Name', how='left')
ind1['value'] = ind1['Total Area of Loss (ha)'] - ind1['Total Area Restorable (ha)']
ind1['unit']='ha'
ind1.drop(columns=['Total Area of Loss (ha)', 'Total Area Restorable (ha)'], inplace = True)
ind1.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,degraded_area,0.152885,ha
1,Angola,AGO,2029,degraded_area,689.862428,ha
2,Anguilla,AIA,1915,degraded_area,0.000000,ha
3,Antigua and Barbuda,ATG,3095,degraded_area,0.000000,ha
4,Aruba,ABW,2707,degraded_area,2.701563,ha


In [26]:
ind1[ind1['Name'] == 'New Zealand']

,Name,iso,id,indicator,value,unit
74,New Zealand,NZL,3648,degraded_area,1333.396209,ha


In [27]:
# Create lost area indicator
ind2 = rp_locs.copy()
ind2['indicator'] = 'lost_area'
ind2 = pd.merge(ind2,data_df[['Name','Total Area of Loss (ha)']],on='Name', how='left').rename(columns={'Total Area of Loss (ha)':'value'})
ind2['unit']='ha'
ind2.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,lost_area,0.594951,ha
1,Angola,AGO,2029,lost_area,1546.820441,ha
2,Anguilla,AIA,1915,lost_area,1.571377,ha
3,Antigua and Barbuda,ATG,3095,lost_area,107.809050,ha
4,Aruba,ABW,2707,lost_area,8.345105,ha


In [28]:
# Add mangrove area indicator
ind3 = rp_locs.copy()
ind3['indicator'] = 'mangrove_area'
ind3 = pd.merge(ind3,data_df[['Name','Total Mangrove Area 2020 (ha)']],on='Name', how='left').rename(columns={'Total Mangrove Area 2020 (ha)':'value'})
ind3['unit']='ha'
ind3.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,mangrove_area,32.004317,ha
1,Angola,AGO,2029,mangrove_area,28319.919690,ha
2,Anguilla,AIA,1915,mangrove_area,3.836882,ha
3,Antigua and Barbuda,ATG,3095,mangrove_area,871.859248,ha
4,Aruba,ABW,2707,mangrove_area,47.371025,ha


In [29]:
# Restorable area indicator
ind4 = rp_locs.copy()
ind4['indicator'] = 'restorable_area'
ind4 = pd.merge(ind4,data_df[['Name','Total Area Restorable (ha)']],on='Name', how='left').rename(columns={'Total Area Restorable (ha)':'value'})
ind4['unit']='ha'
ind4.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,restorable_area,0.442066,ha
1,Angola,AGO,2029,restorable_area,856.958013,ha
2,Anguilla,AIA,1915,restorable_area,1.571377,ha
3,Antigua and Barbuda,ATG,3095,restorable_area,107.809050,ha
4,Aruba,ABW,2707,restorable_area,5.643541,ha


In [30]:
# Create main loss driver indicator
ind5 = rp_locs.copy()
ind5['indicator'] = 'main_loss_driver'
ind5 = pd.merge(ind5,data_df[['Name','Driver']],on='Name', how='left').rename(columns={'Driver':'main_loss_driver'})
ind5= ind5.replace({'main_loss_driver':{'No Data':'undefined'}})
ind5.loc[ind5.main_loss_driver.isna(), 'main_loss_driver'] = 'undefined'
ind5 = ind5[['iso', 'main_loss_driver']]
ind5.head(8)

,iso,main_loss_driver
0,ASM,Commodities
1,AGO,Non-Productive Conversion
2,AIA,Non-Productive Conversion
3,ATG,Non-Productive Conversion
4,ABW,Non-Productive Conversion
5,AUS,Extreme Weather
6,BHS,Extreme Weather
7,BHR,Non-Productive Conversion


In [31]:
ind5.main_loss_driver.value_counts()

Non-Productive Conversion    66
Commodities                  26
Extreme Weather              18
undefined                    12
Name: main_loss_driver, dtype: int64

In [32]:
# Merge all indicators in same table
degradation = pd.concat([ind1, ind2, ind3,ind4], ignore_index=True)
degradation_driver = pd.merge(degradation, ind5, on='iso', how = 'left')
degradation_driver['year'] = 2020
degradation_driver

,Name,iso,id,indicator,value,unit,main_loss_driver,year
0,American Samoa,ASM,2346,degraded_area,0.152885,ha,Commodities,2020
1,Angola,AGO,2029,degraded_area,689.862428,ha,Non-Productive Conversion,2020
2,Anguilla,AIA,1915,degraded_area,0.000000,ha,Non-Productive Conversion,2020
3,Antigua and Barbuda,ATG,3095,degraded_area,0.000000,ha,Non-Productive Conversion,2020
4,Aruba,ABW,2707,degraded_area,2.701563,ha,Non-Productive Conversion,2020
...,...,...,...,...,...,...,...,...
483,Vietnam,VNM,1681,restorable_area,15828.649600,ha,Commodities,2020
484,British Virgin Islands,VGB,3037,restorable_area,24.404751,ha,Non-Productive Conversion,2020
485,"Virgin Islands, U.S.",VIR,2339,restorable_area,54.611497,ha,Non-Productive Conversion,2020
486,Wallis and Futuna,WLF,4170,restorable_area,0.000000,ha,undefined,2020


In [33]:
#Degraded area values calculation might lead to NA values that translate poorly to Int during final format
degradation_driver.loc[degradation_driver.value.isna(),'value'] = 0
print(len(degradation_driver))
print(len(degradation_driver[degradation_driver.value.notnull()]))


488
488


Table cleaning


In [34]:
degradation_final = degradation_driver[degradation_driver.value.notnull()][['id', 'indicator','year', 'value', 'unit', 'main_loss_driver']].copy()
#degradation_final.value = np.int64(degradation_final.value)
degradation_final.rename(columns={'id':'location_id'}, inplace=True)
degradation_final.unit= 'ha'
degradation_final.head(3)

,location_id,indicator,year,value,unit,main_loss_driver
0,2346,degraded_area,2020,0.152885,ha,Commodities
1,2029,degraded_area,2020,689.862428,ha,Non-Productive Conversion
2,1915,degraded_area,2020,0.000000,ha,Non-Productive Conversion


### Data check

In [35]:
sel_country = "Angola"
degradation_driver.query('Name == @sel_country')

,Name,iso,id,indicator,value,unit,main_loss_driver,year
1,Angola,AGO,2029,degraded_area,689.862428,ha,Non-Productive Conversion,2020
123,Angola,AGO,2029,lost_area,1546.820441,ha,Non-Productive Conversion,2020
245,Angola,AGO,2029,mangrove_area,28319.919690,ha,Non-Productive Conversion,2020
367,Angola,AGO,2029,restorable_area,856.958013,ha,Non-Productive Conversion,2020


In [36]:
display(degradation_driver[['indicator', 'value']].groupby('indicator').sum('value'))

,value
indicator,
degraded_area,3.561300e+05
lost_area,1.174439e+06
mangrove_area,1.472742e+07
restorable_area,8.183090e+05


In [37]:
print(f'Initial number of countries is: {len(data_df)}')
print('\nFinal number of countries per indicator:')
display(degradation_driver[['id', 'indicator']].groupby('indicator').count())

Initial number of countries is: 122

Final number of countries per indicator:


,id
indicator,
degraded_area,122
lost_area,122
mangrove_area,122
restorable_area,122


In [38]:
total_loss_original = sum(data_df['Total Area of Loss (ha)'])
total_loss_final = sum(degradation_final[degradation_final['indicator'] == 'lost_area']['value'])
restorable_original = sum(data_df[data_df['Total Area Restorable (ha)'].notna()]['Total Area Restorable (ha)'])
restorable_final = sum(degradation_final[degradation_final['indicator'] == 'restorable_area']['value'])
degraded_original = total_loss_original - restorable_original
degraded_final = sum(degradation_final[degradation_final['indicator'] == 'degraded_area']['value'])

tl =  total_loss_original == total_loss_final
re =  restorable_original == restorable_final 
de =  degraded_original == degraded_final


print('Original and final numbers check')
if(tl):
    print(f'Total loss: {tl}')
else:
    print(f'\tOriginal loss: {total_loss_original}')
    print(f'\tFinal loss: {total_loss_final}')

if(re):
    print(f'Restorable: {re}')
else:
    print(f'\tOriginal restorable: {restorable_original}')
    print(f'\tFinal restorable: {restorable_final}')

if(de):
    print(f'Degradation: {de}')
else:
    print(f'Degradation: {de}')
    print(f'\tOriginal degraded: {degraded_original}')
    print(f'\tFinal degraded: {degraded_final}')


Original and final numbers check
Total loss: True
Restorable: True
Degradation: False
	Original degraded: 356130.0482875712
	Final degraded: 356130.0482875711


Save table

In [42]:
degradation_final[~degradation_final.location_id.isna()].to_csv('../../../../data/Degradation_202212.csv', index = False)

## API Checks

### Restoration data

In [40]:
restoration_df = pd.read_csv('../../../../data/RestorationPotential_202212.csv')
restoration_df.head()

,location_id,indicator,value,unit,year
0,2346,restorable_area,0.442066,ha,2020
1,2029,restorable_area,856.958013,ha,2020
2,1915,restorable_area,1.571377,ha,2020
3,3095,restorable_area,107.809050,ha,2020
4,2707,restorable_area,5.643541,ha,2020


In [ ]:
restoration_endpoint = 'https://mangrove-atlas-api-staging.herokuapp.com/api/v2/widgets/restoration-potential'
d = requests.get(restoration_endpoint)
d.json()['data']
api_restoration = pd.DataFrame(d.json())
api_restoration.reset_index(inplace=True)
api_restoration.rename(columns={'index':'indicator', 'data':'api_value'}, inplace=True)
api_restoration.drop(columns=['metadata'], inplace=True)
api_restoration = api_restoration[0:4]
api_restoration

,indicator,api_value
0,restoration_potential_score,6.800000e+01
1,restorable_area,8.183090e+05
2,restorable_area_perc,5.556365e+00
3,mangrove_area_extent,1.472742e+07


In [43]:
restoration_summary = restoration_df[['indicator', 'value']].groupby('indicator').sum()
restoration_summary.reset_index(inplace=True)
restoration_summary.rename(columns={'index':'indicator', 'value':'file_total'}, inplace=True)
restoration_summary['indicator'] = restoration_summary['indicator'].str.replace('mangrove_area', 'mangrove_area_extent')
restoration_summary

,indicator,file_total
0,mangrove_area_extent,1.472742e+07
1,restorable_area,8.183090e+05
2,restoration_potential_score,7.877000e+03


In [44]:
restoration_combined = pd.merge(restoration_summary, api_restoration, on = 'indicator', how='inner')
#restoration_combined['check'] = round(restoration_combined['file_total'],2) == round(restoration_combined['value'],2)
restoration_combined['check'] = round(restoration_combined['file_total'],2) - round(restoration_combined['api_value'],2)
restoration_combined

## Note: restoration_potential_score is a provided static value, not calculated

,indicator,file_total,api_value,check
0,mangrove_area_extent,1.472742e+07,1.472742e+07,0.0
1,restorable_area,8.183090e+05,8.183090e+05,0.0
2,restoration_potential_score,7.877000e+03,6.800000e+01,7809.0


### Degradation data

In [22]:
degradation_df = pd.read_csv('../../../../data/Degradation_202212.csv')
degradation_df.head()

,location_id,indicator,year,value,unit,main_loss_driver
0,2346,degraded_area,2020,0.152885,ha,Commodities
1,2029,degraded_area,2020,689.862428,ha,Non-Productive Conversion
2,1915,degraded_area,2020,0.000000,ha,Non-Productive Conversion
3,3095,degraded_area,2020,0.000000,ha,Non-Productive Conversion
4,2707,degraded_area,2020,2.701563,ha,Non-Productive Conversion


In [35]:
degradation_summary = degradation_df[['indicator', 'value']].groupby('indicator').sum()
degradation_summary.reset_index(inplace=True)
degradation_summary.rename(columns={'index':'indicator', 'value':'file_total'}, inplace=True)
degradation_summary


,indicator,file_total
0,degraded_area,3.561300e+05
1,lost_area,1.174439e+06
2,mangrove_area,1.472742e+07
3,restorable_area,8.183090e+05


In [32]:
degradation_endpoint = 'https://mangrove-atlas-api-staging.herokuapp.com/api/v2/widgets/degradation-and-loss'

d = requests.get(degradation_endpoint)
api_degradation = pd.DataFrame(d.json()['data'])
api_degradation


,indicator,value,label
0,mangrove_area,1.472742e+07,Mangrove area
1,lost_area,1.174439e+06,Total loss
2,degraded_area,3.561300e+05,Non-Restorable loss mangrove area
3,restorable_area,8.183090e+05,Restorable loss mangrove area


In [42]:
degradation_combined = pd.merge(degradation_summary, api_degradation, on = 'indicator', how='left')
#degradation_combined['check'] = round(degradation_combined['file_total'],2) == round(degradation_combined['value'],2)
degradation_combined['check'] = round(degradation_combined['file_total'],2) - round(degradation_combined['value'],2)
degradation_combined

,indicator,file_total,value,label,check
0,degraded_area,3.561300e+05,3.561300e+05,Non-Restorable loss mangrove area,0.00
1,lost_area,1.174439e+06,1.174439e+06,Total loss,0.44
2,mangrove_area,1.472742e+07,1.472742e+07,Mangrove area,0.00
3,restorable_area,8.183090e+05,8.183090e+05,Restorable loss mangrove area,0.00
